In [2]:
# ================================================================
# BAY AREA DISTRIBUTIONAL IMPACT OF A VMT FEE
# FINAL CLEAN SCRIPT — ALL BUGS FIXED — GOOGLE COLAB READY
# ================================================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
import os

# ================================================================
# 1. CONFIGURATION & CONSTANTS
# ================================================================

DATA_FILE = "df_cleanest+mpg.csv"

# === California Fuel Taxes (per gallon) ===
FEDERAL_TAX_GASOLINE = 0.184
FEDERAL_TAX_DIESEL   = 0.244

STATE_TAX_GASOLINE = 0.612   # CA State Fuel Tax (Gasoline)
STATE_TAX_DIESEL   = 0.466   # CA State Fuel Tax (Diesel)

# Approx retail fuel prices
FUEL_PRICE_GASOLINE = 4.00
FUEL_PRICE_DIESEL   = 4.50

# Base VMT fee
BASE_VMT_FEE = 0.02  # $0.02 per mile

# Income group mapping (Texas method)
INCOME_GROUPS = {
    "low":       [1,2,3,4],
    "mid_low":   [5,6,7],
    "mid_high":  [8,9],
    "high":      [10,11],
}

# ================================================================
# 2. HELPER FUNCTIONS
# ================================================================

def load_and_clean_data(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"DATA FILE NOT FOUND: {file_path}")

    df = pd.read_csv(file_path)

    needed = ["sampno","city","perno","vehno","urbrur","hhfaminc",
              "veh_year","fuel_type_nrel","vmt_mile","mpg_std"]

    for c in needed:
        if c not in df.columns:
            raise ValueError(f"MISSING COLUMN: {c}")

    df = df.dropna(subset=needed)
    df = df[(df["vmt_mile"] > 0) & (df["mpg_std"] > 0)]

    df["sampno"]   = df["sampno"].astype(int)
    df["hhfaminc"] = df["hhfaminc"].astype(int)
    df["urbrur"]   = df["urbrur"].astype(int)

    return df


def classify_income_group(hhinc):
    for group_name, cats in INCOME_GROUPS.items():
        if hhinc in cats:
            return group_name
    return "unknown"


def classify_mpg_class(mpg, q1, q2):
    if mpg <= q1:
        return "low"
    elif mpg <= q2:
        return "mid"
    else:
        return "high"


def get_fuel_price_and_tax(fuel_type):
    t = str(fuel_type).lower()

    if "gas" in t:
        return (FUEL_PRICE_GASOLINE, STATE_TAX_GASOLINE, FEDERAL_TAX_GASOLINE)

    elif "diesel" in t:
        return (FUEL_PRICE_DIESEL, STATE_TAX_DIESEL, FEDERAL_TAX_DIESEL)

    else:
        return (np.nan, 0.0, 0.0)  # EVs


def gini_coefficient(x):
    x = np.array(x, dtype=float)
    if np.all(x == 0):
        return 0.0
    x_sorted = np.sort(x)
    n = len(x_sorted)
    cum = np.cumsum(x_sorted)
    return (2 * np.sum((np.arange(1, n+1) * x_sorted))) / (n*np.sum(x_sorted)) - (n+1)/n


# ================================================================
# 3. SCENARIO 1 — BASELINE FUEL TAX (CURRENT SYSTEM)
# ================================================================

def compute_baseline_fuel_tax(df):
    df = df.copy()

    prices = df["fuel_type_nrel"].apply(get_fuel_price_and_tax)
    df["fuel_price_per_gal"] = [p[0] for p in prices]
    df["state_tax_per_gal"]  = [p[1] for p in prices]
    df["federal_tax_per_gal"]= [p[2] for p in prices]

    df["gallons"] = df["vmt_mile"] / df["mpg_std"]
    df.loc[df["fuel_price_per_gal"].isna(), "gallons"] = 0.0

    df["state_fuel_tax_paid"] = df["gallons"] * df["state_tax_per_gal"]

    hh = df.groupby("sampno").agg(
        hh_total_vmt=("vmt_mile","sum"),
        hh_total_gallons=("gallons","sum"),
        hh_state_fuel_tax_paid=("state_fuel_tax_paid","sum"),
        hh_income_cat=("hhfaminc","max"),
        hh_urbrur=("urbrur","max"),
        hh_city=("city", lambda x: x.mode().iat[0]),
        hh_num_vehicles=("vehno","nunique"),
        hh_avg_mpg=("mpg_std","mean")
    ).reset_index()

    hh["hh_income_group"] = hh["hh_income_cat"].apply(classify_income_group)
    return df, hh


# ================================================================
# 4. COMPUTE CPM + ELASTICITIES
# ================================================================

def compute_household_cpm(df, hh):

    df = df.copy()

    df["cpm_vehicle"] = df["fuel_price_per_gal"] / df["mpg_std"]
    df.loc[df["cpm_vehicle"].isna(), "cpm_vehicle"] = 0.0

    # Weighted CPM: SUM(cpm * vmt) / SUM(vmt)
    cost = df["cpm_vehicle"] * df["vmt_mile"]

    hh_cost = df.groupby("sampno").agg(
        total_cost=("cpm_vehicle", lambda x: np.sum(cost.loc[x.index])),
        total_vmt=("vmt_mile","sum")
    ).reset_index()

    hh_cost["hh_cpm"] = hh_cost["total_cost"] / hh_cost["total_vmt"]

    hh = hh.merge(hh_cost[["sampno","hh_cpm"]], on="sampno", how="left")
    return hh


def estimate_vmt_elasticity(hh):
    hh = hh.copy()

    hh = hh[(hh["hh_total_vmt"] > 0) & (hh["hh_cpm"] > 0)]

    hh["ln_vmt"] = np.log(hh["hh_total_vmt"])
    hh["ln_cpm"] = np.log(hh["hh_cpm"])
    hh["ln_inc"] = np.log(hh["hh_income_cat"])
    hh["ln_veh"] = np.log(hh["hh_num_vehicles"])
    hh["urban_dummy"] = (hh["hh_urbrur"] == 1).astype(int)

    hh["ln_cpm_x_ln_inc"] = hh["ln_cpm"] * hh["ln_inc"]

    X = hh[["ln_cpm","ln_inc","ln_cpm_x_ln_inc","urban_dummy","ln_veh"]]
    X = sm.add_constant(X)
    y = hh["ln_vmt"]

    model = sm.OLS(y, X).fit()

    b1 = model.params["ln_cpm"]
    b3 = model.params["ln_cpm_x_ln_inc"]
    hh["elasticity"] = b1 + b3 * hh["ln_inc"]

    return model, hh


# ================================================================
# 5. SCENARIO 2 — STATIC VMT FEE
# ================================================================

def assign_vmt_fee_rate(df):
    df = df.copy()
    q1 = df["mpg_std"].quantile(1/3)
    q2 = df["mpg_std"].quantile(2/3)

    df["mpg_class"] = df["mpg_std"].apply(lambda x: classify_mpg_class(x, q1, q2))
    df["vmt_fee_rate"] = df["mpg_class"].map({"low":1.2,"mid":1.0,"high":0.8}) * BASE_VMT_FEE
    return df


def compute_vmt_fee_static(df):
    df = df.copy()

    df["vmt_fee_paid_static"] = df["vmt_mile"] * df["vmt_fee_rate"]

    hh = df.groupby("sampno").agg(
        hh_vmt_fee_static=("vmt_fee_paid_static","sum"),
        hh_total_vmt=("vmt_mile","sum"),
        hh_income_cat=("hhfaminc","max"),
        hh_urbrur=("urbrur","max")
    ).reset_index()

    hh["hh_income_group"] = hh["hh_income_cat"].apply(classify_income_group)
    return df, hh


# ================================================================
# 6. SCENARIOS 3 & 4 — DYNAMIC VMT
# ================================================================

def apply_dynamic_vmt(df, hh_baseline, hh_elasticity, zero_positive=False):

    df = df.copy()

    # FIXED MERGE: only 'elasticity'
    hh = hh_baseline.merge(
        hh_elasticity[["sampno","elasticity"]],
        on="sampno", how="left"
    )

    # Scenario 3 option
    if zero_positive:
        hh.loc[hh["elasticity"] > 0, "elasticity"] = 0.0

    eps = 1e-9

    hh["state_tax_cpm"] = hh["hh_state_fuel_tax_paid"] / (hh["hh_total_vmt"] + eps)
    hh["cpm_new"] = hh["hh_cpm"] - hh["state_tax_cpm"]

    hh["delta_cpm"] = hh["cpm_new"] - hh["hh_cpm"]
    hh["rel_change_cpm"] = hh["delta_cpm"] / (hh["hh_cpm"] + eps)

    hh["hh_total_vmt_dyn"] = hh["hh_total_vmt"] * (
        1 + hh["elasticity"] * hh["rel_change_cpm"]
    )

    df = df.merge(hh[["sampno","hh_total_vmt","hh_total_vmt_dyn"]], on="sampno", how="left")

    df["scale"] = df["hh_total_vmt_dyn"] / (df["hh_total_vmt"] + eps)
    df["vmt_mile_dyn"] = df["vmt_mile"] * df["scale"]

    df["vmt_fee_paid_dynamic"] = df["vmt_mile_dyn"] * df["vmt_fee_rate"]

    hh_dyn = df.groupby("sampno").agg(
        hh_vmt_fee_dynamic=("vmt_fee_paid_dynamic","sum"),
        hh_total_vmt_dyn=("vmt_mile_dyn","sum"),
        hh_income_cat=("hhfaminc","max"),
        hh_urbrur=("urbrur","max")
    ).reset_index()

    hh_dyn["hh_income_group"] = hh_dyn["hh_income_cat"].apply(classify_income_group)
    return df, hh_dyn


# ================================================================
# 7. GINI SUMMARY
# ================================================================

def compute_gini_results(h1, h2, h3, h4):

    scenarios = [
        ("Scenario 1: Fuel Tax", "baseline", h1, "hh_state_fuel_tax_paid"),
        ("Scenario 2: Static VMT", "static", h2, "hh_vmt_fee_static"),
        ("Scenario 3: Dyn VMT (mod)", "dyn_mod", h3, "hh_vmt_fee_dynamic"),
        ("Scenario 4: Dyn VMT (raw)", "dyn_raw", h4, "hh_vmt_fee_dynamic"),
    ]

    rows = []

    for label, code, hh, col in scenarios:
        rows.append({
            "scenario": label,
            "code": code,
            "gini_total": gini_coefficient(hh[col]),
            "gini_urban": gini_coefficient(hh.loc[hh["hh_urbrur"]==1, col]),
            "gini_rural": gini_coefficient(hh.loc[hh["hh_urbrur"]!=1, col])
        })

    return pd.DataFrame(rows)


# ================================================================
# 8. MAIN PIPELINE
# ================================================================

print("Loading dataset...")
df = load_and_clean_data(DATA_FILE)

print("Scenario 1 (Fuel Tax)...")
df1, hh1 = compute_baseline_fuel_tax(df)

print("Computing CPM...")
hh1 = compute_household_cpm(df1, hh1)

print("Estimating elasticity...")
model, hh_elast = estimate_vmt_elasticity(hh1)
print(model.summary())

print("Elasticity summary:")
print(hh_elast.groupby("hh_income_group")["elasticity"].mean())

print("Assigning VMT fee...")
df_fee = assign_vmt_fee_rate(df1)

print("Scenario 2 (Static VMT)...")
df2, hh2 = compute_vmt_fee_static(df_fee)

print("Scenario 3 (Dynamic Mod)...")
df3, hh3 = apply_dynamic_vmt(df_fee, hh1, hh_elast, zero_positive=True)

print("Scenario 4 (Dynamic Raw)...")
df4, hh4 = apply_dynamic_vmt(df_fee, hh1, hh_elast, zero_positive=False)

print("Computing Gini...")
gini = compute_gini_results(hh1, hh2, hh3, hh4)
print(gini)

print("Saving outputs...")
hh1.to_csv("scenario1_fuel_tax.csv", index=False)
hh2.to_csv("scenario2_static_vmt.csv", index=False)
hh3.to_csv("scenario3_dynamic_modified.csv", index=False)
hh4.to_csv("scenario4_dynamic_raw.csv", index=False)
gini.to_csv("gini_summary.csv", index=False)

print("\n✔ DONE — all scenarios computed and files saved.")


Loading dataset...
Scenario 1 (Fuel Tax)...
Computing CPM...
Estimating elasticity...
                            OLS Regression Results                            
Dep. Variable:                 ln_vmt   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.231
Method:                 Least Squares   F-statistic:                     56.02
Date:                Thu, 11 Dec 2025   Prob (F-statistic):           8.12e-51
Time:                        09:17:41   Log-Likelihood:                -1644.7
No. Observations:                 915   AIC:                             3301.
Df Residuals:                     909   BIC:                             3330.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------